<a href="https://colab.research.google.com/github/Mahaveersalvih/movie-recommendation-/blob/main/Copy_of_Project_18_Mood_based_Movie_Recommendation_System_using_Machine_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [2]:
!pip install streamlit
import numpy as np
import pandas as pd
import streamlit as st
import requests
import pickle
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('/content/movies.csv')

In [4]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [5]:
# number of rows and columns in the data frame

movies_data.shape

(4803, 24)

In [6]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director','id']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'id']


In [7]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [8]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']+' '+movies_data['id'].astype(str)

In [9]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [10]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [12]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 129068 stored elements and shape (4803, 22079)>
  Coords	Values
  (0, 4962)	0.07558463332496819
  (0, 5035)	0.08675091800727598
  (0, 10035)	0.10682369557117992
  (0, 18360)	0.0996650735240196
  (0, 10198)	0.0996650735240196
  (0, 8439)	0.20571442887524974
  (0, 7826)	0.21356327043425335
  (0, 10597)	0.15835713517009034
  (0, 19139)	0.32659730933843956
  (0, 21348)	0.12067958495774306
  (0, 7986)	0.24002536695991955
  (0, 19032)	0.20571442887524974
  (0, 9706)	0.23104072162345685
  (0, 20022)	0.0682359348921808
  (0, 21759)	0.12329360255434745
  (0, 15953)	0.08702131936522213
  (0, 16264)	0.2616731586704858
  (0, 18110)	0.14444955478119179
  (0, 21768)	0.227362220585816
  (0, 22051)	0.19422995678876445
  (0, 18080)	0.20939292991289057
  (0, 18825)	0.1980589697405917
  (0, 21429)	0.1908195360387445
  (0, 19369)	0.1456939887959868
  (0, 13517)	0.21837757524935328
  :	:
  (4801, 22027)	0.2762414077101103
  (4801, 18596)	0.26675

Cosine Similarity

In [13]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [14]:
print(similarity)

[[1.         0.06736808 0.03483192 ... 0.         0.         0.        ]
 [0.06736808 1.         0.03056728 ... 0.03330229 0.         0.        ]
 [0.03483192 0.03056728 1.         ... 0.         0.04952055 0.        ]
 ...
 [0.         0.03330229 0.         ... 1.         0.         0.02447526]
 [0.         0.         0.04952055 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02447526 0.         1.        ]]


In [15]:
print(similarity.shape)

(4803, 4803)


Getting the movie name from the user

In [16]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : batman


In [17]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [18]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Batman', 'Batman', 'Catwoman']


In [19]:
close_match = find_close_match[0]
print(close_match)

Batman


In [20]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

1359


In [21]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.02360241097438915)), (1, np.float64(0.046883622075585434)), (2, np.float64(0.012657168809010706)), (3, np.float64(0.18881474855674898)), (4, np.float64(0.02328854247842329)), (5, np.float64(0.10723269677159626)), (6, np.float64(0.0)), (7, np.float64(0.005144704343802425)), (8, np.float64(0.02876977882218632)), (9, np.float64(0.08377554326531828)), (10, np.float64(0.10377329600358652)), (11, np.float64(0.01109752244893349)), (12, np.float64(0.035613196368072184)), (13, np.float64(0.012436854377902045)), (14, np.float64(0.11953861851951347)), (15, np.float64(0.009759461006451718)), (16, np.float64(0.004897226185204527)), (17, np.float64(0.01495896669403498)), (18, np.float64(0.038856146673999614)), (19, np.float64(0.027939756615545167)), (20, np.float64(0.02351178089665008)), (21, np.float64(0.0043290520390913715)), (22, np.float64(0.016827298558908348)), (23, np.float64(0.02341112804684125)), (24, np.float64(0.03919563187656229)), (25, np.float64(0.0)), (26, np.float64

In [22]:
len(similarity_score)

4803

In [23]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(1359, np.float64(1.0000000000000002)), (428, np.float64(0.40027175742504967)), (210, np.float64(0.24077013824181)), (3, np.float64(0.18881474855674898)), (119, np.float64(0.17887189075199894)), (65, np.float64(0.16529835133986678)), (1512, np.float64(0.13856436013270895)), (813, np.float64(0.13489551731799077)), (1017, np.float64(0.12967884553357595)), (2530, np.float64(0.1284085259744972)), (753, np.float64(0.12360464922913761)), (473, np.float64(0.12344899431701381)), (278, np.float64(0.12221720074908687)), (14, np.float64(0.11953861851951347)), (72, np.float64(0.11461962448013263)), (2381, np.float64(0.11059547086918159)), (2313, np.float64(0.10979159531982388)), (4183, np.float64(0.1075901192678338)), (5, np.float64(0.10723269677159626)), (2858, np.float64(0.10579042092791274)), (30, np.float64(0.105629970265678)), (2655, np.float64(0.10455251900685783)), (1035, np.float64(0.10426126125006904)), (10, np.float64(0.10377329600358652)), (2685, np.float64(0.10328421692484961)), (1296

In [26]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<11):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Batman
2 . Batman Returns
3 . Batman & Robin
4 . The Dark Knight Rises
5 . Batman Begins
6 . The Dark Knight
7 . A History of Violence
8 . Superman
9 . Bedazzled
10 . Beetlejuice


Movie Recommendation Sytem

In [28]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<20):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : superman
Movies suggested for you : 

1 . Superman
2 . Superman II
3 . Superman IV: The Quest for Peace
4 . Man of Steel
5 . Superman III
6 . Crimson Tide
7 . Superman Returns
8 . Suicide Squad
9 . Batman Returns
10 . The Dark Knight Rises
11 . The Killer Inside Me
12 . Nanny McPhee and the Big Bang
13 . Batman Begins
14 . The Dark Knight
15 . The Godfather
16 . Batman
17 . Batman
18 . Batman & Robin
19 . The Island of Dr. Moreau


In [29]:
import pickle
with open('movies.pkl','wb')as file:
    pickle.dump(movies_data,file)

with open('similarity.pkl','wb')as file:
    pickle.dump(similarity,file)

In [30]:
mood_to_genres = {
    'happy': ['Comedy', 'Family', 'Animation', 'Adventure'],
    'sad': ['Drama', 'Tragedy', 'Romance'],
    'action': ['Action', 'Adventure', 'Science Fiction', 'Thriller'],
    'romantic': ['Romance', 'Drama'],
    'thrilling': ['Thriller', 'Action', 'Crime', 'Mystery'],
    'family-friendly': ['Family', 'Animation', 'Comedy', 'Adventure'],
    'dark': ['Thriller', 'Crime', 'Mystery', 'Horror', 'Science Fiction'],
    'funny': ['Comedy', 'Animation', 'Family']
}

print("Mood to genre mapping dictionary created:")
print(mood_to_genres)

Mood to genre mapping dictionary created:
{'happy': ['Comedy', 'Family', 'Animation', 'Adventure'], 'sad': ['Drama', 'Tragedy', 'Romance'], 'action': ['Action', 'Adventure', 'Science Fiction', 'Thriller'], 'romantic': ['Romance', 'Drama'], 'thrilling': ['Thriller', 'Action', 'Crime', 'Mystery'], 'family-friendly': ['Family', 'Animation', 'Comedy', 'Adventure'], 'dark': ['Thriller', 'Crime', 'Mystery', 'Horror', 'Science Fiction'], 'funny': ['Comedy', 'Animation', 'Family']}


In [31]:
user_mood = input("Enter your current mood (e.g., happy, sad, action, romantic, thrilling, family-friendly, dark, funny): ")
print(f"You entered: {user_mood}")

Enter your current mood (e.g., happy, sad, action, romantic, thrilling, family-friendly, dark, funny): sad
You entered: sad


In [32]:
genres_for_mood = mood_to_genres.get(user_mood.lower(), [])

if not genres_for_mood:
    print(f"Mood '{user_mood}' not recognized. Please choose from: {', '.join(mood_to_genres.keys())}.")
    print("No genres retrieved for this mood.")
else:
    print(f"Genres associated with '{user_mood}': {genres_for_mood}")

Genres associated with 'sad': ['Drama', 'Tragedy', 'Romance']


In [33]:
import pandas as pd
import random

movies_data = pd.read_csv('/content/movies.csv')
movies_data['genres'] = movies_data['genres'].fillna('') # Fill NaN values with empty strings
mood_based_recommendations = []

if genres_for_mood:
    for index, row in movies_data.iterrows():
        movie_genres = row['genres'].split(' ')
        if any(genre in movie_genres for genre in genres_for_mood):
            mood_based_recommendations.append(index)

# Shuffle the recommendations to get different movies each time
random.shuffle(mood_based_recommendations)

print(f"Total movies found for '{user_mood}': {len(mood_based_recommendations)}")
print("First 10 recommended movie indices for this mood (shuffled):")
for i, movie_index in enumerate(mood_based_recommendations[:10]):
    title = movies_data.loc[movie_index, 'title']
    print(f"  {i+1}. {title} (Index: {movie_index})")

Total movies found for 'sad': 2586
First 10 recommended movie indices for this mood (shuffled):
  1. Raising Helen (Index: 962)
  2. The Longshots (Index: 2005)
  3. Play It to the Bone (Index: 1979)
  4. Stealing Harvard (Index: 1908)
  5. Saved! (Index: 3614)
  6. The Hunting Party (Index: 1282)
  7. Travellers and Magicians (Index: 4182)
  8. Stonewall (Index: 4122)
  9. Beneath Hill 60 (Index: 3205)
  10. Fly Me to the Moon (Index: 1919)


In [34]:
print(f"\nTop 15 Movies for your '{user_mood}' mood:")

if mood_based_recommendations:
    for i, movie_index in enumerate(mood_based_recommendations[:15]):
        title = movies_data.loc[movie_index, 'title']
        print(f"  {i+1}. {title}")
else:
    print("No movies found for this mood.")


Top 15 Movies for your 'sad' mood:
  1. Raising Helen
  2. The Longshots
  3. Play It to the Bone
  4. Stealing Harvard
  5. Saved!
  6. The Hunting Party
  7. Travellers and Magicians
  8. Stonewall
  9. Beneath Hill 60
  10. Fly Me to the Moon
  11. Dragon Blade
  12. The Deep End of the Ocean
  13. The Mothman Prophecies
  14. Redemption Road
  15. America's Sweethearts


In [35]:
# 1. Install Streamlit
!pip install -q streamlit

# 2. Get your public IP (You will need this password shortly)
!wget -q -O - ipv4.icanhazip.com

136.118.191.229


In [37]:
# 3. Run Streamlit in the background and expose it via localtunnel
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://136.118.191.229:8501

your url is: https://little-taxes-dig.loca.lt
  Stopping...
^C


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
